In [140]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D, Conv1D, LSTM
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

import string
from nltk.tokenize import word_tokenize 
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()



In [141]:
from numpy import array
from numpy import asarray
from numpy import zeros

In [2]:
df = pd.read_csv("./train.csv")
df = df[:150000]
df.shape

(150000, 6)

In [35]:
def preprocess_text(ques):
    ques = str(ques)
    ques = [str(word) for word in ques]
    ques = "".join([word.lower() for word in ques if word not in string.punctuation])
    ques = word_tokenize(ques)
    ques = [ps.stem(word) for word in ques]
    ques = [str(word) for word in ques]
    ques = " ".join(ques)
    return ques
    

In [36]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [37]:
'''X_q1 = []
X_q2 = []
q1 = list(df['question1'])
q2 = list(df['question2'])
for ques in q1:
    X_q1.append(preprocess_text(ques))
for ques in q2:
    X_q2.append(preprocess_text(ques))'''

In [122]:
X = []
for ind,row in df.iterrows():
    X.append(preprocess_text(str(row['question1']) + " " + str(row['question2'])))

In [38]:
'''X = pd.DataFrame()
X['question1'] = X_q1
X['question2'] = X_q2'''

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, df['is_duplicate'], test_size=0.20, random_state=42)

In [86]:
'''tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(list(X_train['question1']))
X_train_q1 = tokenizer.texts_to_sequences(list(X_train['question1']))
X_test_q1 = tokenizer.texts_to_sequences(list(X_test['question1']))

tokenizer.fit_on_texts(list(X_train['question2']))
X_train_q2 = tokenizer.texts_to_sequences(list(X_train['question2']))
X_test_q2 = tokenizer.texts_to_sequences(list(X_test['question2']))'''

In [124]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(list(X_train))
X_test = tokenizer.texts_to_sequences(list(X_test))

In [87]:
'''vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train_q1 = pad_sequences(X_train_q1, padding='post', maxlen=maxlen)
X_test_q1 = pad_sequences(X_test_q1, padding='post', maxlen=maxlen)

X_train_q2 = pad_sequences(X_train_q2, padding='post', maxlen=maxlen)
X_test_q2 = pad_sequences(X_test_q2, padding='post', maxlen=maxlen)'''

In [125]:
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [81]:
embeddings = {}
f = open(r"./glove.6B.100d.txt", encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],dtype=np.float32)
    embeddings[word] = vector

In [126]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    try:
        embedding_vector = embeddings[word]
        embedding_matrix[index] = embedding_vector
    except: 
        pass

In [127]:
vocab_size

45265

In [128]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [129]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          4526500   
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 10001     
Total params: 4,536,501
Trainable params: 10,001
Non-trainable params: 4,526,500
_________________________________________________________________
None


In [116]:
X_train = []
for i in range(len(X_train_q1)):
    arr = []
    arr.extend(X_train_q1[i])
    arr.extend(X_train_q2[i])
    X_train.append(arr)


In [117]:
X_test = []
for i in range(len(X_test_q1)):
    arr = []
    arr.extend(X_test_q1[i])
    arr.extend(X_test_q2[i])
    X_test.append(arr)


In [118]:
X_train = np.array(X_train,dtype=np.float32)
X_test = np.array(X_test,dtype=np.float32)

In [130]:
history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

Epoch 1/6
750/750 [==============================] - 5s 6ms/step - loss: 0.6116 - acc: 0.6636 - val_loss: 0.6142 - val_acc: 0.6717
Epoch 2/6
750/750 [==============================] - 5s 6ms/step - loss: 0.5900 - acc: 0.6839 - val_loss: 0.6031 - val_acc: 0.6772
Epoch 3/6
750/750 [==============================] - 5s 7ms/step - loss: 0.5835 - acc: 0.6867 - val_loss: 0.6026 - val_acc: 0.6801
Epoch 4/6
750/750 [==============================] - 4s 6ms/step - loss: 0.5807 - acc: 0.6895 - val_loss: 0.6099 - val_acc: 0.6686
Epoch 5/6
750/750 [==============================] - 5s 7ms/step - loss: 0.5792 - acc: 0.6896 - val_loss: 0.6109 - val_acc: 0.6699
Epoch 6/6
750/750 [==============================] - 5s 6ms/step - loss: 0.5773 - acc: 0.6923 - val_loss: 0.6249 - val_acc: 0.6520


In [131]:
score = model.evaluate(X_test, y_test, verbose=1)

938/938 [==============================] - 3s 3ms/step - loss: 0.6223 - acc: 0.6545


In [132]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.6222784519195557
Test Accuracy: 0.6545000076293945


### CNN

In [135]:
model = Sequential()

embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)

model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [136]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 100)          4526500   
_________________________________________________________________
conv1d (Conv1D)              (None, 96, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 4,590,757
Trainable params: 64,257
Non-trainable params: 4,526,500
_________________________________________________________________
None


In [137]:
history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=1)

Epoch 1/6
750/750 [==============================] - 46s 61ms/step - loss: 0.5644 - acc: 0.7068 - val_loss: 0.5430 - val_acc: 0.7249
Epoch 2/6
750/750 [==============================] - 43s 57ms/step - loss: 0.5001 - acc: 0.7539 - val_loss: 0.5083 - val_acc: 0.7488
Epoch 3/6
750/750 [==============================] - 46s 61ms/step - loss: 0.4611 - acc: 0.7791 - val_loss: 0.4946 - val_acc: 0.7567
Epoch 4/6
750/750 [==============================] - 44s 59ms/step - loss: 0.4268 - acc: 0.7981 - val_loss: 0.4915 - val_acc: 0.7614
Epoch 5/6
750/750 [==============================] - 44s 58ms/step - loss: 0.3965 - acc: 0.8171 - val_loss: 0.4959 - val_acc: 0.7582
Epoch 6/6
938/938 [==============================] - 5s 6ms/step - loss: 0.5332 - acc: 0.7496


In [138]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.5331975817680359
Test Accuracy: 0.7495666742324829


In [142]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(LSTM(128))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [143]:
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 100)          4526500   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 4,643,877
Trainable params: 117,377
Non-trainable params: 4,526,500
_________________________________________________________________
None


In [144]:
history = model.fit(X_train, y_train, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=1)

Epoch 1/6
750/750 [==============================] - 136s 182ms/step - loss: 0.6357 - acc: 0.6399 - val_loss: 0.6010 - val_acc: 0.6742
Epoch 2/6
750/750 [==============================] - 145s 194ms/step - loss: 0.5901 - acc: 0.6858 - val_loss: 0.5738 - val_acc: 0.7036
Epoch 3/6
750/750 [==============================] - 137s 183ms/step - loss: 0.5664 - acc: 0.7092 - val_loss: 0.5579 - val_acc: 0.7166
Epoch 4/6
750/750 [==============================] - 131s 175ms/step - loss: 0.5460 - acc: 0.7241 - val_loss: 0.5508 - val_acc: 0.7222
Epoch 5/6
750/750 [==============================] - 130s 173ms/step - loss: 0.5284 - acc: 0.7367 - val_loss: 0.5353 - val_acc: 0.7316
Epoch 6/6
938/938 [==============================] - 36s 38ms/step - loss: 0.5255 - acc: 0.7335


In [145]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.525536298751831
Test Accuracy: 0.7334666848182678
